<a href="https://colab.research.google.com/github/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/feature_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 117.2 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 1.02GB/s]


In [6]:
! unzip walmart-recruiting-store-sales-forecasting.zip

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Install required packages
import subprocess
import sys

In [8]:
!pip install prophet plotly mlflow dagshub xgboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [12]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.xgboost
import dagshub
import xgboost as xgb
import joblib
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import os
warnings.filterwarnings('ignore')

In [14]:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)



In [15]:
class WalmartPreprocessingPipeline:
    def __init__(self, train_date_split='2012-04-13', remove_outliers=True, enable_lag_features=False):
        self.train_date_split = pd.to_datetime(train_date_split)
        self.remove_outliers = remove_outliers
        self.enable_lag_features = enable_lag_features
        self.weekly_sales_outlier_thresholds = {
            'A': {'lower': -20000, 'upper': 75000},
            'B': {'lower': -10000, 'upper': 40000},
            'C': {'lower': -5000, 'upper': 20000}
        }
        self.outliers_removed_count = 0

    def fit_transform(self, train_df, features_df, stores_df):
        """
        Main method to merge, clean, engineer features, and prepare data.
        MarkDown features are now INCLUDED.
        """
        print("📊 Starting data preprocessing...")
        df = self._merge_data(train_df, features_df, stores_df)
        df = self._clean_data(df)
        df = self._feature_engineer_dates(df)
        df = self._feature_engineer_holidays(df)
        df = self._feature_engineer_store_type(df)
        df = self._handle_markdown_nans(df) # NEW: Handle NaNs in MarkDowns
        if self.remove_outliers:
            df = self._remove_outliers(df)
        if not self.enable_lag_features:
            df = self._remove_lag_features(df) # This will no longer remove MarkDowns
        df = self._handle_missing_values(df) # General NaN handling
        df = self._remove_redundant_features(df)
        print("✅ Data preprocessing complete.")
        return df

    def _merge_data(self, train_df, features_df, stores_df):
        print("   Merging datasets...")
        df = pd.merge(train_df, stores_df, on='Store', how='left')
        df = pd.merge(df, features_df, on=['Store', 'Date'], how='left')
        print(f"   Merged data shape: {df.shape}")
        return df

    def _clean_data(self, df):
        print("   Cleaning merged data (handling IsHoliday duplicates)...")
        if 'IsHoliday_x' in df.columns and 'IsHoliday_y' in df.columns:
            df['IsHoliday'] = df['IsHoliday_x'] | df['IsHoliday_y']
            df = df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
        df['IsHoliday'] = df['IsHoliday'].astype(bool)
        return df

    def _feature_engineer_dates(self, df):
        print("   Creating date features...")
        df['Date'] = pd.to_datetime(df['Date'])
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)
        df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
        df['IsMonthStart'] = (df['Date'].dt.day == 1).astype(int)
        df['IsMonthEnd'] = (df['Date'].dt.is_month_end).astype(int)

        min_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - min_date).dt.days
        df['WeeksFromStart'] = (df['DaysFromStart'] // 7).astype(int)
        return df

    def _feature_engineer_holidays(self, df):
        print("   Creating holiday features...")
        holidays = {
            'SuperBowl': [datetime(y, 2, d) for y in range(2010, 2013) for d in [7, 8, 9, 10, 11, 12, 13]],
            'LaborDay': [datetime(y, 9, d) for y in range(2010, 2013) for d in [1, 2, 3, 4, 5, 6, 7]],
            'Thanksgiving': [datetime(y, 11, d) for y in range(2010, 2013) for d in [22, 23, 24, 25, 26, 27, 28]],
            'Christmas': [datetime(y, 12, d) for y in range(2010, 2013) for d in [24, 25, 26, 27, 28, 29, 30, 31]]
        }

        df['IsSuperBowlWeek'] = df['Date'].isin(holidays['SuperBowl']).astype(int)
        df['IsLaborDayWeek'] = df['Date'].isin(holidays['LaborDay']).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].isin(holidays['Thanksgiving']).astype(int)
        df['IsChristmasWeek'] = df['Date'].isin(holidays['Christmas']).astype(int)

        df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] | df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)
        df['IsHolidayMonth'] = df['Month'].isin([11, 12, 1, 2, 9]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8]).astype(int)
        return df

    def _feature_engineer_store_type(self, df):
        print("   Encoding store Type...")
        df['Type_Encoded'] = df['Type'].astype('category').cat.codes
        df = pd.get_dummies(df, columns=['Type'], prefix='Type', drop_first=False)
        return df

    def _handle_markdown_nans(self, df):
        """
        Handles NaN values in MarkDown columns.
        MarkDowns are often NaN when no markdown was applied.
        Filling with 0 is a common strategy.
        """
        print("   Handling NaNs in MarkDown columns (filling with 0)...")
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        for col in markdown_cols:
            df[col] = df[col].fillna(0)
        return df

    def _remove_outliers(self, df):
        print("   Removing outliers from Weekly_Sales based on store type...")
        initial_rows = len(df)
        df_cleaned = pd.DataFrame()
        for store_type in self.weekly_sales_outlier_thresholds.keys():
            if f'Type_{store_type}' in df.columns:
                subset = df[df[f'Type_{store_type}'] == 1].copy()
                lower = self.weekly_sales_outlier_thresholds[store_type]['lower']
                upper = self.weekly_sales_outlier_thresholds[store_type]['upper']
                subset = subset[(subset['Weekly_Sales'] >= lower) & (subset['Weekly_Sales'] <= upper)]
                df_cleaned = pd.concat([df_cleaned, subset])
            else:
                pass # If Type_A, Type_B, or Type_C don't exist

        if not df_cleaned.empty:
            self.outliers_removed_count = initial_rows - len(df_cleaned)
            if self.outliers_removed_count > 0:
                print(f"   🗑️ Removed {self.outliers_removed_count} outliers.")
            return df_cleaned
        else:
            print("   Warning: Outlier removal resulted in an empty DataFrame. Returning original DataFrame.")
            return df

    def _remove_lag_features(self, df):
        # This method is kept for consistency with previous pipelines,
        # but since MarkDowns are now included, it won't remove them.
        # If any other specific 'lagged_sales' features were created, they'd be removed here.
        return df

    def _handle_missing_values(self, df):
        print("   Handling remaining missing values (filling numerical NaNs with 0)...")
        numerical_cols = df.select_dtypes(include=np.number).columns
        df[numerical_cols] = df[numerical_cols].fillna(0)
        return df

    def _remove_redundant_features(self, df):
        print("   Removing redundant features...")
        cols_to_drop = [
            'Year', # Often redundant with other temporal features
            'Day',  # Covered by DayOfWeek, Month
            # 'Size' is kept as it's a useful feature
        ]
        return df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors='ignore')


In [18]:
# --- Main Execution for Feature Importance Analysis ---
def main():
    print("🚀 Starting Feature Importance Analysis with MarkDowns...")

    # Load data (assuming data is unzipped in the current directory)
    print("Loading raw data...")
    # Ensure raw data files (train.csv, features.csv, stores.csv) are in the same directory
    # or provide the correct paths.
    try:
        train_raw = pd.read_csv('train.csv')
        features_raw = pd.read_csv('features.csv')
        stores_raw = pd.read_csv('stores.csv')
        print("Data loaded successfully.")
    except FileNotFoundError:
        print("Data files not found. Attempting to unzip...")
        try:
            with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            with zipfile.ZipFile('stores.csv.zip', 'r') as zip_ref: # Assuming stores.csv might also be zipped
                zip_ref.extractall('.')
            train_raw = pd.read_csv('train.csv')
            features_raw = pd.read_csv('features.csv')
            stores_raw = pd.read_csv('stores.csv')
            print("Data unzipped and loaded successfully.")
        except Exception as e:
            print(f"Error loading or unzipping data: {e}")
            print("Please ensure 'train.csv', 'features.csv', 'stores.csv' (or their .zip files) are in the current directory.")
            return


    # Initialize and run the preprocessing pipeline
    # MarkDowns are now included by default as _remove_markdowns is not called in fit_transform
    pipeline = WalmartPreprocessingPipeline(
        remove_outliers=True,
        enable_lag_features=False # Lag features are still off, but MarkDowns are separate
    )
    processed_df = pipeline.fit_transform(train_raw, features_raw, stores_raw)
    print(f"\nProcessed data shape: {processed_df.shape}")
    print(f"Columns in processed data: {processed_df.columns.tolist()}")

    # --- Feature Importance Analysis (using Correlation) ---
    print("\n📊 Analyzing Feature Importance based on Correlation with Weekly_Sales...")

    # Define target variable
    target_col = 'Weekly_Sales'

    # Exclude non-feature columns (like 'Date', 'Store', 'Dept' if not treated as numerical features for correlation)
    # For correlation, we typically want numerical features.
    # Store and Dept are categorical IDs, their direct correlation might not be meaningful
    # but their encoded versions (if any) or one-hot versions could be.
    # For this simple correlation, let's include all numerical-like features.

    # Ensure 'Date' is not in features for correlation
    features_for_correlation = [col for col in processed_df.columns if col not in [target_col, 'Date']]

    # Calculate correlations
    correlations = processed_df[features_for_correlation + [target_col]].corr()[target_col].drop(target_col)

    # Convert to absolute values for importance (magnitude of correlation)
    abs_correlations = correlations.abs().sort_values(ascending=False)

    print("\n--- Feature Correlations with Weekly_Sales (Absolute Value, Sorted) ---")
    print(abs_correlations)

    # Define a threshold to categorize features
    # This threshold is subjective; adjust based on domain knowledge or further analysis
    correlation_threshold = 0.06 # Example threshold: features with abs correlation >= 0.05 are 'important'

    important_features = abs_correlations[abs_correlations >= correlation_threshold].index.tolist()
    less_important_features = abs_correlations[abs_correlations < correlation_threshold].index.tolist()

    print(f"\n--- Categorized Feature Importance (Threshold: {correlation_threshold}) ---")
    print("\n✅ Important Features (Absolute Correlation >= {}):".format(correlation_threshold))
    if important_features:
        for feature in important_features:
            print(f"   - {feature}: {abs_correlations[feature]:.4f}")
    else:
        print("   No features met the importance threshold.")

    print("\n❌ Less Important Features (Absolute Correlation < {}):".format(correlation_threshold))
    if less_important_features:
        for feature in less_important_features:
            print(f"   - {feature}: {abs_correlations[feature]:.4f}")
    else:
        print("   All features met the importance threshold.")

    print("\n🎉 Feature Importance Analysis Completed!")

if __name__ == '__main__':
    # Unzip data if necessary (assuming they are in the same directory)
    # This block is for convenience in environments like Colab
    try:
        # Check if raw files exist, if not, try unzipping
        if not os.path.exists('train.csv') and os.path.exists('train.csv.zip'):
            with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if not os.path.exists('features.csv') and os.path.exists('features.csv.zip'):
            with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if not os.path.exists('stores.csv') and os.path.exists('stores.csv.zip'):
            with zipfile.ZipFile('stores.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
    except Exception as e:
        print(f"Error during initial zip extraction check: {e}")
        print("Please ensure data files (or their zips) are in the current directory.")



In [19]:
main()

🚀 Starting Feature Importance Analysis with MarkDowns...
Loading raw data...
Data loaded successfully.
📊 Starting data preprocessing...
   Merging datasets...
   Merged data shape: (421570, 17)
   Cleaning merged data (handling IsHoliday duplicates)...
   Creating date features...
   Creating holiday features...
   Encoding store Type...
   Handling NaNs in MarkDown columns (filling with 0)...
   Removing outliers from Weekly_Sales based on store type...
   🗑️ Removed 28279 outliers.
   Handling remaining missing values (filling numerical NaNs with 0)...
   Removing redundant features...
✅ Data preprocessing complete.

Processed data shape: (393291, 34)
Columns in processed data: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday', 'Month', 'DayOfWeek', 'WeekOfYear', 'IsWeekend', 'IsMonthStart', 'IsMonthEnd', 'DaysFromStart', 'WeeksFromStart', 'IsSuperBowlWeek'